### Build Docker Image that contains Resnet 152 model and Flask web application
Make sure you are able to run Docker without sudo.  
Make sure you are have logged in using docker login. 


In [7]:
import os
from os import path
import json

In [8]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

mkdir: cannot create directory ‘flaskwebapp’: File exists
mkdir: cannot create directory ‘flaskwebapp/nginx’: File exists
mkdir: cannot create directory ‘flaskwebapp/etc’: File exists


In [9]:
!cp model.h5 flaskwebapp
#!cp synset.txt flaskwebapp
!ls flaskwebapp

3layers10epochs.h5  etc			   model.h5
app.py		    gunicorn_logging.conf  nginx
dockerfile	    kill_supervisor.py	   requirements.txt


Below is the module for the Flask web application.

In [47]:
%%writefile flaskwebapp/app.py

# import the necessary packages
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image,ImageOps
import numpy as np
import flask
import io

# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None

classes=['axes','boots','carabiners','crampons','gloves','hardshell_jackets','harnesses','helmets','insulated_jackets','pulleys','rope','tents']

def load_our_model():
    # load the pre-trained Keras model (here we are using a model
    # pre-trained on ImageNet and provided by Keras, but you can
    # substitute in your own networks just as easily)
    global model
    model = load_model('model.h5')
    model._make_predict_function()

def prepare_image(im, size=128, fill_color=(255,255,255)): #take a PIL image
    im.thumbnail((size,size), Image.ANTIALIAS)
    x, y = im.size    
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    new_im=ImageOps.equalize(new_im)
    im_array = img_to_array(new_im)
    im_array=np.expand_dims(im_array, axis=0)
    #imagenet_utils.preprocess_input(im_array)
    return im_array

@app.route("/predict", methods=["POST"])
def predict():
    # initialize the data dictionary that will be returned from the
    # view
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))

            # preprocess the image and prepare it for classification
            image = prepare_image(image)

            # classify the input image and then initialize the list
            # of predictions to return to the client
            preds = model.predict_classes(image)
            #results = imagenet_utils.decode_predictions(preds)
            data["prediction"] = classes[int(preds)]

            # loop over the results and add them to the list of
            # returned predictions
            #for (imagenetID, label, prob) in results[0]:
            #    r = {"label": label, "probability": float(prob)}
            #    data["predictions"].append(r)

            # indicate that the request was a success
            data["success"] = True

    # return the data dictionary as a JSON response
    return flask.jsonify(data)

# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    load_our_model()
    app.run(debug=True,host='0.0.0.0')

Overwriting flaskwebapp/app.py


In [48]:
%%writefile flaskwebapp/requirements.txt
keras
numpy
pillow
Flask
tensorflow

Overwriting flaskwebapp/requirements.txt


In [49]:
image_name = "kaizimmerm/challenge5"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

We create a custom image based on Ubuntu 16.04 and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [50]:
%%writefile flaskwebapp/dockerfile

FROM ubuntu:16.04

RUN mkdir /code
WORKDIR /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        openmpi-bin \
        python3 \ 
        python3-dev \ 
        python3-setuptools \
        python3-pip && \
    pip3 install -r /code/requirements.txt

EXPOSE 5000

ENTRYPOINT ["python3"]
CMD ["app.py"]

Overwriting flaskwebapp/dockerfile


In [51]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon  10.08MB
Step 1/8 : FROM ubuntu:16.04
 ---> b9e15a5d1e1a
Step 2/8 : RUN mkdir /code
 ---> Running in af82aa74e3d5
Removing intermediate container af82aa74e3d5
 ---> 8ec77b09b991
Step 3/8 : WORKDIR /code
 ---> Running in e0b7e0c1beb8
Removing intermediate container e0b7e0c1beb8
 ---> b2c712c177bd
Step 4/8 : ADD . /code/
 ---> daf44d14eff5
Step 5/8 : RUN apt-get update && apt-get install -y --no-install-recommends         openmpi-bin         python3         python3-dev         python3-setuptools         python3-pip &&     pip3 install -r /code/requirements.txt
 ---> Running in 1a6e2b60f13c
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-backports InRelease [107 kB]
Get:4 http://archive.ubuntu.com/ubuntu xenial/universe Sources [9802 kB]
Get:5 http://security.ubuntu.com/ubuntu xenial-security InRelease [107 kB]
Get:6

Selecting previously unselected package libexpat1:amd64.
Preparing to unpack .../libexpat1_2.1.0-7ubuntu0.16.04.3_amd64.deb ...
Unpacking libexpat1:amd64 (2.1.0-7ubuntu0.16.04.3) ...
Selecting previously unselected package python3.5-minimal.
Preparing to unpack .../python3.5-minimal_3.5.2-2ubuntu0~16.04.4_amd64.deb ...
Unpacking python3.5-minimal (3.5.2-2ubuntu0~16.04.4) ...
Selecting previously unselected package python3-minimal.
Preparing to unpack .../python3-minimal_3.5.1-3_amd64.deb ...
Unpacking python3-minimal (3.5.1-3) ...
Selecting previously unselected package mime-support.
Preparing to unpack .../mime-support_3.59ubuntu1_all.deb ...
Unpacking mime-support (3.59ubuntu1) ...
Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../libmpdec2_2.4.2-1_amd64.deb ...
Unpacking libmpdec2:amd64 (2.4.2-1) ...
Selecting previously unselected package libsqlite3-0:amd64.
Preparing to unpack .../libsqlite3-0_3.11.0-1ubuntu1_amd64.deb ...
Unpacking libsqlite3-0:amd6

Setting up libpython3.5:amd64 (3.5.2-2ubuntu0~16.04.4) ...
Setting up libpython3.5-dev:amd64 (3.5.2-2ubuntu0~16.04.4) ...
Setting up libpython3-dev:amd64 (3.5.1-3) ...
Setting up libhwloc5:amd64 (1.11.2-3) ...
Setting up libibverbs1 (1.1.8-1.1ubuntu2) ...
Setting up libopenmpi1.10 (1.10.2-8ubuntu1) ...
Setting up openmpi-common (1.10.2-8ubuntu1) ...
Setting up openmpi-bin (1.10.2-8ubuntu1) ...
update-alternatives: using /usr/bin/mpirun.openmpi to provide /usr/bin/mpirun (mpirun) in auto mode
Setting up python-pip-whl (8.1.1-2ubuntu0.4) ...
Setting up python3.5-dev (3.5.2-2ubuntu0~16.04.4) ...
Setting up python3 (3.5.1-3) ...
running python rtupdate hooks for python3.5...
running python post-rtupdate hooks for python3.5...
Setting up python3-dev (3.5.1-3) ...
Setting up python3-pip (8.1.1-2ubuntu0.4) ...
Setting up python3-pkg-resources (20.7.0-1) ...
Setting up python3-setuptools (20.7.0-1) ...
Setting up dh-python (2.20151103ubuntu1.1) ...
Processing triggers for libc-bin (2.23-0ubunt

  Running setup.py bdist_wheel for itsdangerous: finished with status 'error'
  Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-svujkn19/itsdangerous/setup.py';exec(compile(getattr(tokenize, 'open', open)(__file__).read().replace('\r\n', '\n'), __file__, 'exec'))" bdist_wheel -d /tmp/tmplsmhsf7epip-wheel- --python-tag cp35:
  usage: -c [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: -c --help [cmd1 cmd2 ...]
     or: -c --help-commands
     or: -c cmd --help
  
  error: invalid command 'bdist_wheel'
  
  ----------------------------------------
  Failed building wheel for itsdangerous
  Running setup.py clean for itsdangerous
  Running setup.py bdist_wheel for absl-py: started
  Running setup.py bdist_wheel for absl-py: finished with status 'error'
  Complete output from command /usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-svujkn19/absl-py/setup.py';exec(compile(getattr(tokenize, 'o

In [52]:
!docker push $image_name

The push refers to repository [docker.io/kaizimmerm/challenge5]

5a930ba8: Preparing 
78d98a86: Preparing 
f141f585: Preparing 
9e19929c: Preparing 
b2f378bb: Preparing 
afdbe580: Preparing 
43c86cbc: Preparing 
5a930ba8: Pushed   827.9MB/814.7MBntu Klatest: digest: sha256:60b3a89630fba5d1cb1bde26e97f49bb2e9b685712e3ff16664cc49c41608f42 size: 1987


In [53]:
print('Docker image name {}'.format(image_name)) 

Docker image name kaizimmerm/challenge5


In [16]:
!curl -X post -F image=@gear_images/hardshell_jackets/ortovox-civetta-hardshell.jpg http://challenge5.westeurope.azurecontainer.io:5000/predict

{
  "prediction": "hardshell_jackets", 
  "success": true
}


### Test locally
Go to the [Test Locally notebook](TestLocally.ipynb) to test your Docker image